In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI私有终点

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/tree/main/notebooks/official/prediction/sdk_private_endpoint.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub标志">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/prediction/sdk_private_endpoint.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI标志">
      在Vertex AI工作台中打开
    </a>
  </td>                                                                                               
</table>

**请确保此笔记本在VPC内运行，因为`PrivateEndpoint`预测方法只能在私有网络中执行。创建一个Vertex AI工作台实例，并在该实例中上传和运行此笔记本。在创建新的Vertex AI工作台实例时，请使用`default`子网或按照[此处](https://cloud.google.com/vpc/docs/create-modify-vpc-networks)中概述的步骤创建和使用唯一的VPC。**

## 概述

本教程演示了如何使用Vertex AI SDK创建和使用Vertex AI `PrivateEndpoint`资源来提供模型服务。`PrivateEndpoint`提供了一个低延迟、安全、私密的网络连接到Vertex AI在线预测服务（即企业内部网络）。这消除了公共`Endpoint`（即互联网）的网络交换和路由的开销。

了解更多关于[`PrivateEndpoint`资源](https://cloud.google.com/vertex-ai/docs/predictions/using-private-endpoints)。

### 数据集

本教程使用公共Cloud Storage存储桶`gs://cloud-samples-data/ai-platform-unified/datasets/tabular/`中的[petfinder](https://storage.googleapis.com/cloud-samples-data/ai-platform-unified/datasets/tabular/petfinder-tabular-classification-tabnet-with-header.csv)数据集，该数据集源自[PetFinder.my Adoption Prediction](https://www.kaggle.com/c/petfinder-adoption-prediction)。该数据集预测动物被领养的速度。

### 目标

在本教程中，您将学习如何使用`Vertex AI PrivateEndpoint`资源。

本教程使用以下Google Cloud Platform Vertex AI服务和资源：

- `Vertex AI TabNet`
- `Vertex AI TrainingJob`
- `Vertex AI Model`
- `Vertex AI PrivateEndpoint`
- `Vertex AI Prediction`

执行的步骤包括：

- 导入训练数据。
- 为`Vertex AI TabNet`模型容器配置训练参数。
- 使用`Vertex AI TrainingJob`训练CSV数据。
- 将模型上传为`Vertex AI Model`资源。
- 配置VPC对等连接。
- 创建一个`Vertex AI PrivateEndpoint`资源。
- 将`Vertex AI Model`资源部署到`Vertex AI PrivateEndpoint`资源。
- 发送预测请求到`Vertex AI PrivateEndpoint`。
- 清理资源。

### 成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解有关[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

### 安装
安装执行此笔记所需的包。

In [ ]:
! pip3 install --upgrade --quiet tensorflow
! pip3 install --upgrade --quiet google-cloud-aiplatform
! gcloud components update --quiet

仅限协作：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果您有选择的话，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得 $300 的免费信用用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下 API：Vertex AI APIs，Compute Engine APIs 和 Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您是在本地运行此笔记本，您需要安装 [Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，确保 Cloud SDK 为本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 会运行以 `!` 开头的行作为 shell 命令，并会插值以 `$` 开头的 Python 变量。

设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参考支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解更多关于 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动进行身份验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要做任何操作，因为您已经通过身份验证。

**2. 本地 JupyterLab 实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 请查看如何为您的服务账号授予云存储权限，网址为https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

导入库并定义常数

In [ ]:
import os

import google.cloud.aiplatform as aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

### 设置训练容器

接下来，使用预先构建的`Vertex AI TabNet`容器来训练模型。

TabNet结合了两种最好的世界：它是可解释的（类似于简单的基于树的模型），同时又从深度神经网络中获益。这使得它非常适合零售商、金融和保险行业的应用，如预测信用评分、诈骗检测和预测。

TabNet使用一种名为序列关注的机器学习技术，在模型的每一步中选择要进行推理的模型特征。这种机制使得可以解释模型如何做出预测，并帮助它学习更加准确的模型。TabNet不仅胜过其他神经网络和决策树，还提供可解释的特征归因。

阅读研究论文：[TabNet：关注性可解释的表格学习](https://arxiv.org/pdf/1908.07442.pdf)。

In [ ]:
TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai-restricted/builtin-algorithm/tab_net_v2"
print("Training container:", TRAIN_IMAGE)

### 设置预构建容器以便部署

设置预构建的Docker容器镜像用于预测。

获取最新列表，请参见[预构建用于预测的容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2.5".replace(".", "-")

if TF[0] == "2":
    DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment container:", DEPLOY_IMAGE)

获取训练数据

从公共云存储桶中获取训练数据的副本（以CSV文件的形式），并将训练数据复制到您的云存储桶中。

In [ ]:
# Please note that if you use csv input, the first column is the label column.

IMPORT_FILE = "petfinder-tabular-classification-tabnet-with-header.csv"
TRAINING_DATA_PATH = f"{BUCKET_URI}/data/petfinder/train.csv"

! gsutil cp gs://cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {TRAINING_DATA_PATH}

## 训练 Vertex AI TabNet 模型

要训练 TabNet 自定义模型，您需要创建并运行一个自定义训练作业。

### 创建自定义训练作业

使用 `CustomContainerTrainingJob` 类创建自定义训练作业，需要以下参数：

- `display_name`: 自定义训练作业的人类可读名称。
- `container_uri`: 训练容器镜像。
- `model_serving_container_image_uri`: 可为您的模型提供预测的容器的 URI。

In [ ]:
DATASET_NAME = "petfinder"  # Change to your dataset name.

job = aiplatform.CustomContainerTrainingJob(
    display_name=f"{DATASET_NAME}",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

print(job)

### 为 TabNet 训练配置参数设置

通过指南 [使用内置的 TabNet 算法入门](https://cloud.google.com/ai-platform/training/docs/algorithms/tab-net-start) 了解更多关于使用 TabNet 的信息。

In [ ]:
ALGORITHM = "tabnet"
MODEL_TYPE = "classification"
MODEL_NAME = f"{DATASET_NAME}_{ALGORITHM}_{MODEL_TYPE}"

OUTPUT_DIR = f"{BUCKET_URI}/{MODEL_NAME}"
print("Output dir: ", OUTPUT_DIR)

CMDARGS = [
    "--preprocess",
    "--data_has_header",
    f"--training_data_path={TRAINING_DATA_PATH}",
    f"--job-dir={OUTPUT_DIR}",
    f"--model_type={MODEL_TYPE}",
    "--max_steps=2000",
    "--batch_size=4096",
    "--learning_rate=0.01",
    "--prediction_raw_inputs",
    "--exclude_key",
]

运行自定义训练作业并创建TabNet模型

使用`run`方法开始训练，该方法接受以下参数：

- `model_display_name`：如果脚本生成了托管的`Model`，则为`Model`的显示名称。
- `args`：要传递给TabNet训练容器的命令行参数。
- `replica_count`：worker副本的数量。
- `machine_type`：用于训练的机器类型。
- `base_output_dir`：作业的GCS输出目录。
- `sync`：是否同步执行此方法。

`run`方法创建一个训练管道，训练并创建一个`Model`对象。训练管道完成后，`run`方法将返回`Model`对象。

In [ ]:
MODEL_DIR = OUTPUT_DIR
MACHINE_TYPE = "n1-standard-4"

model = job.run(
    model_display_name=f"{DATASET_NAME}",
    args=CMDARGS,
    replica_count=1,
    machine_type=MACHINE_TYPE,
    base_output_dir=MODEL_DIR,
    sync=True,
)

print(model.gca_resource)

### 删除训练作业

使用 `delete()` 方法来删除训练作业。

In [ ]:
job.delete()

建立一个VPC对等连接网络

要使用`PrivateEndpoint`，您需要在您的项目和托管运行模型的Vertex AI预测服务项目之间设置VPC对等网络。 这消除了网络流量中的额外跳跃，并允许使用有效的HTTP协议。

了解更多关于[VPC对等](https://cloud.google.com/vertex-ai/docs/general/vpc-peering)的信息。

**重要提示：每个项目只能设置一个到servicenetworking.googleapis.com的VPC对等连接。**

### 为`default`网络创建VPC peering

为了简单起见，我们设置了VPC peering到新建的GCP（Google Cloud Platform）项目默认使用的`default`网络。您也可以为您的项目创建和使用不同的网络。如果您设置了VPC peering到其他网络，请确保该网络已经存在，并且您的虚拟机正在该网络上运行。

In [ ]:
# This is for display only; you can name the range anything.
PEERING_RANGE_NAME = "vertex-ai-prediction-peering-range"
NETWORK = "default"

# NOTE: `prefix-length=16` means a CIDR block with mask /16 will be
# reserved for use by Google services, such as Vertex AI.
! gcloud compute addresses create $PEERING_RANGE_NAME \
  --global \
  --prefix-length=16 \
  --description="peering range for Google service" \
  --network=$NETWORK \
  --purpose=VPC_PEERING

### 创建VPC连接

为VPC对等连接创建连接。

注意：如果收到“PERMISSION DENIED”错误消息，可能是因为您的默认服务帐号没有设置所需的角色`Compute Network Admin`。在Cloud Console中执行以下步骤。

1. 在GCP仪表板中转到`IAM & Admin`。
2. 找到您的服务账号。
3. 点击编辑图标。
4. 选择添加另一个角色。
5. 输入`Compute Network Admin`。
6. 选择保存。

In [ ]:
! gcloud services vpc-peerings connect \
  --service=servicenetworking.googleapis.com \
  --network=$NETWORK \
  --ranges=$PEERING_RANGE_NAME \
  --project=$PROJECT_ID

检查您对等连接的状态。

In [ ]:
! gcloud compute networks peerings list --network $NETWORK

### 构建完整的网络名称

在随后为VPC peering创建`PrivateEndpoint`资源时，您需要拥有完整的网络资源名称。

In [ ]:
project_number = model.resource_name.split("/")[1]
print(project_number)

full_network_name = f"projects/{project_number}/global/networks/{NETWORK}"
full_network_name

## 创建一个`PrivateEndpoint`资源

使用`PrivateEndpoint.create()`方法创建一个`PrivateEndpoint`资源。

在这个示例中，指定了以下参数：

- `display_name`：`PrivateEndpoint`资源的人类可读的名称。
- `network`：VPC peering的完整网络资源名称。

In [ ]:
if not os.getenv("IS_TESTING"):
    private_endpoint = aiplatform.PrivateEndpoint.create(
        display_name=f"{DATASET_NAME}_private_endpoint",
        network=full_network_name,
    )

获取`PrivateEndpoint`资源的详细信息

查看具有属性`gca_resource`的`PrivateEndpoint`对象的底层详细信息。

In [ ]:
if not os.getenv("IS_TESTING"):
    private_endpoint.gca_resource

将TabNet模型部署到`PrivateEndpoint`中

将TabNet模型部署到新创建的`PrivateEndpoint`资源上，以对传入数据样本进行预测。

该函数接受以下参数：

- `model`: 要部署的模型。
- `deployed_model_display_name`: 用于部署模型的人类可读名称。
- `machine_type`: 用于训练的机器类型。

该方法将阻塞，直到模型被部署，并最终返回一个`PrivateEndpoint`对象。如果这是将模型首次部署到端点，则可能需要额外几分钟来完成资源的供应。

In [ ]:
DEPLOYED_NAME = f"{DATASET_NAME}_deployed_model"

if not os.getenv("IS_TESTING"):
    response = private_endpoint.deploy(
        model=model,
        deployed_model_display_name=DEPLOYED_NAME,
        machine_type="n1-standard-4",
    )

### 获取服务签名

在本地下载模型并查询其服务签名。服务签名的形式如下：

    ( "feature_name_1",  "feature_name_2", ... )

In [ ]:
import tensorflow as tf

loaded = tf.saved_model.load(MODEL_DIR + "/model")
loaded.signatures

最后，使用`predict()`方法进行预测。每个实例都以以下字典格式指定：

{ "feature_name_1": value, "feature_name_2", value, ... }

In [ ]:
if not os.getenv("IS_TESTING"):
    prediction = private_endpoint.predict(
        [
            {
                "Age": 3,
                "Breed1": "Tabby",
                "Color1": "Black",
                "Color2": "White",
                "Fee": 100,
                "FurLength": "Short",
                "Gender": "Male",
                "Health": "Healthy",
                "MaturitySize": "Small",
                "PhotoAmt": 2,
                "Sterilized": "No",
                "Type": "Cat",
                "Vaccinated": "No",
            }
        ]
    )

    print(prediction)

清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除您用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False
try:
    private_endpoint.delete(force=True)
    model.delete()
except Exception as e:
    print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI